<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_demucs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import runes_client
except ImportError:
    !pip install runes-client
    import runes_client
    import runes_client.core as runes
    from runes_client import RunesFilePath


async def register_imports():
    import os
    import subprocess
    import stat
    import time

    # Check if demucs is installed, if not then install it
    try:
        import demucs
    except ImportError:
        !pip install demucs
        import demucs

    # Check if ffmpeg is installed, if not then install it
    ffmpeg_installed = !command -v ffmpeg
    if not ffmpeg_installed:
        !apt-get install ffmpeg
        !chmod +x /usr/bin/ffmpeg

    # Updating globals to include these modules
    globals()['os'] = os
    globals()['subprocess'] = subprocess
    globals()['stat'] = stat
    globals()['time'] = time
    globals()['demucs'] = demucs
    globals()['ffmpeg_installed'] = ffmpeg_installed


runes.register_imports(register_imports)

placeholder_txt = "Enter the token generated by the Crucible plugin"
RUNES_TOKEN = "cd245614-bb77-4b60-9521-2935a94b5da8" #@param {type:"string"}
runes_token = RUNES_TOKEN

if runes_token is None or runes_token == "" or runes_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def runes_func(input_file: RunesFilePath):
    try:
        command = ['python3', '-m', 'demucs']

        print(f"input_file: {input_file}")

        # Add the audio file path
        command.append(input_file)

        # Execute the command
        subprocess.run(command)

        model = 'htdemucs'
        input_file_without_extension = os.path.splitext(os.path.basename(input_file))[0]
        path = os.path.join('separated', model, input_file_without_extension)

        files = os.listdir(path)
        for file in files:
            full_path = os.path.join(path, file)

            await runes.output().add_file(full_path)

        await runes.output().add_message("Stem Seperation Complete")

        return True
    except Exception as e:

        await runes.output().add_error(f"Method encountered an error: {e}")
        return False


runes.set_token(token=runes_token)
runes.set_name("Stem Separation - DEMUCS")
runes.set_description("Send an audio file & have the Rune split stems and return them")
runes.register_method(runes_func)

runes.set_output_target_format('wav')
runes.set_output_target_channels(2)
runes.set_output_target_sample_rate(44100)
runes.set_output_target_bit_depth(16)

runes.connect_to_server()